# Ethereum ENS tutorial and playground with JavaScript & web3.js

Tested on Energy Web Foundation's Tobalaba test network.

Please make a test account with some test tokens for experimentation

In [ ]:
const Web3 = require('web3')
const ENS = require('ethereum-ens')
const namehash = require('eth-ens-namehash')
const fs = require("fs")

const registryAddress = "0x76b4d318cf896288fd44315e93c891ad4017aa14"
const registrarAddress = "0xf106c25e7624726dfa52a25f4fef29ec6a8ce103"
const resolverAddress = "0x1da4c76ed136f32693700714502c6dacd62f5a3c"
const reverseResolverAddress = "0x9d17a1ef45df65af5cdbfc98e3bfe3e90463780e"
const reverseRegistrarAddress = "0x89a3ae0d90304b00cea94c0f5528b1c657b041cc"

const registryAbi = JSON.parse(fs.readFileSync("abis/ENSRegistry.abi"))
const registrarAbi = JSON.parse(fs.readFileSync("abis/FIFSRegistrar.abi"))
const resolverAbi = JSON.parse(fs.readFileSync("abis/PublicResolver.abi"))
const reverseResolverAbi = JSON.parse(fs.readFileSync("abis/DefaultReverseResolver.abi"))
const reverseRegistrarAbi = JSON.parse(fs.readFileSync("abis/ReverseRegistrar.abi"))
const resolverInterfaceAbi = JSON.parse(fs.readFileSync("abis/ResolverInterface.abi"))

const ADDRESS_EMPTY = '0x0000000000000000000000000000000000000000'

In [ ]:
var provider = new Web3.providers.HttpProvider("http://localhost:8545")
var w3 = new Web3(provider)
var sha3 = w3.sha3

// if you want to use an ENS library, you need it to point to the EWF Registry address
var ens = new ENS(w3, registryAddress)

// set default account and unlock it
w3.eth.defaultAccount = w3.eth.accounts[0]
//w3.personal.unlockAccount(w3.eth.defaultAccount, "yourpassowrd")

In [ ]:
// Instantiate the deployed registry and resolver contracts to use
Registry = w3.eth.contract(registryAbi)
Registrar = w3.eth.contract(registrarAbi)
Resolver = w3.eth.contract(resolverAbi)
ReverseResolver = w3.eth.contract(reverseResolverAbi)
ReverseRegistrar = w3.eth.contract(reverseRegistrarAbi)

registry = Registry.at(registryAddress)
registrar = Registrar.at(registrarAddress)
resolver = Resolver.at(resolverAddress)
reverseResolver = ReverseResolver.at(reverseResolverAddress)
reverseRegistrar = ReverseRegistrar.at(reverseRegistrarAddress)

## Doing lookups and reverse lookups

### Resolving names (lookup)

 1. Get resolver address of a node from registry
 2. Instantiate resolver contrac with resolver interface abi
 3. Fetch registered address (or whatever you need)

In [ ]:
var nameToCheck = "mycompany.ewf"
regAddr = registry.resolver.call(namehash.hash(nameToCheck))

// check if resolver is actually set
if(regAddr !== '0x0000000000000000000000000000000000000000') {
    ResolverInterface = w3.eth.contract(resolverInterfaceAbi)
    resInt = ResolverInterface.at(regAddr)
    console.log(resInt.addr.call(namehash.hash(nameToCheck)))
} else {
    console.log("This name cannot be resolved: resolver not set.")
}

### Resolving addresses (reverse lookup)

1. Get resolver address of the address node from registry
2. Instantiate reverse resolver contract
3. Fetch canonical name for the address


In [ ]:
// More explanation below at "Step 4 - Set reverse lookup"
var addressToResolve = w3.eth.accounts[0]
var addressNodeHash = reverseRegistrar.node.call(addressToResolve)

// Fetch the resolver
AReverseResolver = w3.eth.contract(reverseResolverAbi)
aReverseResolver = AReverseResolver.at(reverseResolverAddress)
registeredName = aReverseResolver.name.call(addressNodeHash)

console.log(registeredName + " is the canonical name to " + addressToResolve)

## Registering and managing names (nodes)

### Step 1 - Check if a name is available to register

In [ ]:
// check ownership directly interacting with the registry contract
var nameToCheck = "mycompany.ewf"
if(registry.owner.call(namehash.hash(nameToCheck)) === ADDRESS_EMPTY) {
    console.log(nameToCheck + " is avaiable")
} else {
    console.log(nameToCheck + " is taken")
}

### Step 2 - Registering ownership of a name

Claim ownership of your desired domain (FIFS registrar: first come, first served) It will be registered under the .ewf top level domain

In [ ]:
var nameowner = w3.eth.accounts[0]
var label = 'mycompany'
registrar.register.sendTransaction(sha3(label), nameowner)

In [ ]:
// Let's check if we succeed

takenBy = registry.owner.call(namehash.hash(label + ".ewf"))
if (takenBy === ADDRESS_EMPTY) {
    console.log(label + ".ewf is avaiable")
} else {
    // should be your owner address
    console.log(label + ".ewf is taken by: " + takenBy)
}

#### Change owner of the node

In [ ]:
var newowner = "new_owner_address_here"
registry.setOwner.sendTransaction(namehash.hash(label), newowner)

### Step 3 - Mapping an address to your name
Map the name to point to your desired address in a Resolver contract.

 - Feel free to use the supplied public resolver, but you can also deploy your own
 - You need to have ownership of the name already
 - Give it a lil gas



In [ ]:
var pointToAddress = w3.eth.accounts[0]
var nameToMap = "mycompany.ewf"
txhash = resolver.setAddr.sendTransaction(namehash.hash(nameToMap), pointToAddress, {"gas": 800000})

In [ ]:
// verify the result - wait for like 5 secs till it get mined. No error, no problems
assert(resolver.addr.call(namehash.hash(nameToMap)) == pointToAddress)
console.log("Great!")

#### Change resolver of a node

In [ ]:
// change resolver of a node
var newResolverAddress = "your_resolver_address_here"
registry.setResolver.sendTransaction(namehash.hash(nameToMap), newResolverAddress)

### Step 4 - Set reverse lookup
Now you also want others to check the canonical name that belongs to your address. Multiple names can belong to an address, but you can set only one for reverse lookups. This step is optional.

Reverse Registrar holds the ".addr.reverse" top domain. Address nodes are registered in the form of "< hex account address >.addr.reverse". To reverse lookup the name for an address, just simply do a lookup to < hex account address >.addr.reverse. Don't forget to remove the the leading '0x' from the address.

In [ ]:
// Registering the reverse record

// Calling account must be the owner of the name
var nameOwner = w3.eth.accounts[0]
var nameToMap = "mypreferredname.ewf"
reverseRegistrar.setName.sendTransaction(nameToMap, {"from": nameOwner})

In [ ]:
// Doing a reverse lookup method 1 - getting the address node hash from the registrar contract
var addressnodeNamehash1 = reverseRegistrar.node.call(nameOwner)
var registeredName1 = reverseResolver.name.call(addressnodeNamehash1)

// Doing a reverse lookup method 2 - caculating the addres node hash yourself
// you have to cut down the '0x' beginning of the address string
var addressnodeNamehash2 = namehash.hash(nameOwner.slice(2)+".addr.reverse")
var registeredName2 = reverseResolver.name.call(addressnodeNamehash2)

// Doing a reverse lookup method 3 - if a custom/different resolver was used then the provided one,
// first you can get its address from the registry
var customReverseResolverAddress = registry.resolver.call(addressnodeNamehash1)
// instantiate using its abi, then proceed with a regular lookup as in version 1 or 2
// in this case it is the same abi as before (this is just a showcase)
var CustomReverseResolver = w3.eth.contract(reverseResolverAbi)
var customReverseResolver = CustomReverseResolver.at(customReverseResolverAddress)
var registeredName3 = customReverseResolver.name.call(addressnodeNamehash1)

// See that the 3 methods are equal
assert(registeredName1 === registeredName2 && registeredName2 === registeredName3)

console.log(nameOwner + " reverse lookup: " + registeredName1)

### Step 5 - Register subnodes

Once you have ownership of a name (node), you have the ability to register sub-nodes as well. E.g: adam.yourdomain.ewf You have to be the owner of the root domain (in this case yourdomain.ewf) in order to register a subnode.


In [ ]:
// E.g. I want to register my name (adam) as a subdomain under mycompany.ewf => adam.mycompany.ewf
// in this case I register the subnode for my account as well
var subnodeowner = w3.eth.accounts[0]
registry.setSubnodeOwner.sendTransaction(namehash.hash("mypreferredname.ewf"), sha3("adam"), subnodeowner, {"gas": 800000})

In [ ]:
// verify the ownership transfer - wait for like 5 secs till it get mined
assert(registry.owner.call(namehash.hash("adam.mypreferredname.ewf")) === subnodeowner)
console.log("Works!")

### Step 6 - Your experiments here

Feel free to play around with ENS